In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os

In [2]:
df_i_antro= pd.read_spss('data/Anthropometry-Caspian.sav',convert_categoricals=True)
df_i_next = pd.read_spss('data/caspian1 data.sav',convert_categoricals=True)
df_i_prev = pd.read_spss('data/CASPIANI.sav',convert_categoricals=True) 

def rename_features(df1,is_new=False):
    df2 = df1.copy()
    # Rename the features in the dataframe
    if is_new:
        df2.columns = df2.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType', 'wc':'waist'}, inplace=True)      
    else:
        df2.columns = df2.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)      
    return df2

df_i_prev=rename_features(df_i_prev)
df_i_antro=rename_features(df_i_antro)
df_i_next=rename_features(df_i_next)

# Now define df_dict with the modified dataframes


In [3]:
df_i_prev.columns

Index(['id', 'code', 'sex', 'university', 'state', 'province', 'region',
       'schoolType', 'grade', 'birthdat',
       ...
       'tg98', 'hdl50', 'fbs100', 'wc75', 'dbp_sbp2', 'tg150', 'hdl35', 'wc2',
       'ms_a', 'ms_b'],
      dtype='object', length=318)

In [4]:
df_i_antro.columns

Index(['id', 'code', 'sex', 'university', 'state', 'province', 'region',
       'schoolType', 'grade', 'birthdat', 'age', 'fjob', 'feducati', 'mjob',
       'meducati', 'househol', 'housing', 'weight', 'height', 'waist', 'hip',
       'bmi', 'fbs_rec', 'tc_r', 'tg_rec', 'tc_rec', 'ldl_re', 'hdl_rec',
       'bmi_rec'],
      dtype='object')

In [1]:
a = df_i_antro.university.unique()

NameError: name 'df_i_antro' is not defined

In [6]:
with open('university_unique2.txt', 'w') as f:
    for item in a:
        f.write(f"{item}\n")

In [7]:
df_dict = {'caspian_I_prev': df_i_prev,'caspian_I_antro': df_i_antro, 'caspian_I_next': df_i_next}

def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["weight", "height", "sex", "age"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["height", "weight", "sex"])
        print(
            f"Number of records with NaN value in weight or height in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs

processed_dfs = preprocess(df_dict)
# Define your mappings dictionary for each Caspian dataset
mappings = {
    'caspian_I_prev': {
        "Gorgan": "Golestan",
        "Mashad": "Razavi Khorasan",
        "ShahidBeheshti": "Tehran",
        "Tabriz": "East Azerbaijan",
        "Yazd": "Yazd",
        "Rasht": "Gilan"
    },
    'caspian_I_antro':{
            "Rasht": "Gilan",
            "Arak": "Markazi",
            "Ahvaz": "Khuzestan",
            "Tabriz": "East Azerbaijan",
            "Khoram_Abad": "Lorestan",
            "Zahedan": "Sistan and Baluchestan",
            "Zabol": "Sistan and Baluchestan",
            "Semnan": "Semnan",
            "Sanandaj": "Kurdistan",
            "Gorgan": "Golestan",
            "Mashad": "Razavi Khorasan",
            "Yazd": "Yazd",
            "Ealam": "Ilam",
            "Qom": "Qom",
            "Kermanshah": "Kermanshah",
            "Kerman": "Kerman",
            "Oroomieh": "West Azerbaijan",
            "Ardebil": "Ardabil",
            "Bandar_Abbas": "Hormozgan",
            "Booshehr": "Bushehr",
            "Zanjan": "Zanjan",
            "ShahrKord": "Chaharmahal and Bakhtiari",
            "ShahidBeheshti": "Tehran"},
        
}
university_to_province = mappings.get('caspian_I_prev')
processed_dfs['caspian_I_prev']['university'] = processed_dfs['caspian_I_prev']['university'].replace(university_to_province)           
# Ensure the 'university' column is of type string
processed_dfs['caspian_I_prev']['university'] = processed_dfs['caspian_I_prev']['university'].astype(str)
# Sort the DataFrame by the 'university' column
processed_dfs['caspian_I_prev'] = processed_dfs['caspian_I_prev'].sort_values(by='university', ascending=True)

university_to_province = mappings.get('caspian_I_antro')
processed_dfs['caspian_I_antro']['university'] = processed_dfs['caspian_I_antro']['university'].replace(university_to_province)           
# Ensure the 'university' column is of type string
processed_dfs['caspian_I_antro']['university'] = processed_dfs['caspian_I_antro']['university'].astype(str)
# Sort the DataFrame by the 'university' column
processed_dfs['caspian_I_antro'] = processed_dfs['caspian_I_antro'].sort_values(by='university', ascending=True)

#

Number of records with NaN value in weight or height in caspian_I_prev: 11
Number of records with NaN value in weight or height in caspian_I_antro: 132
Number of records with NaN value in weight or height in caspian_I_next: 132


/tmp/ipykernel_137271/3365927308.py:67: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  processed_dfs['caspian_I_prev']['university'] = processed_dfs['caspian_I_prev']['university'].replace(university_to_province)
/tmp/ipykernel_137271/3365927308.py:74: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  processed_dfs['caspian_I_antro']['university'] = processed_dfs['caspian_I_antro']['university'].replace(university_to_province)


In [8]:
df1=processed_dfs['caspian_I_antro']
df2=processed_dfs['caspian_I_next']

column_to_exclude = "id"

# Select all columns except the specified one
matching_features = [col for col in df1.columns if col != column_to_exclude]

# Merge the datasets on matching features
merged_df = pd.merge(df1, df2, on=matching_features, suffixes=('_df1', '_df2'))

# Group by 'id2' from df2 and collect 'id2' values from df1
result = merged_df.groupby('id_df2')['id_df1'].apply(list).reset_index()

# Rename columns for clarity
result.columns = ['record_id_df2', 'records_id_df1']


# Step 1: Flatten the list of `id2` values from `records_id2_df1`
id2_to_remove = set([item for sublist in result['records_id_df1'] for item in sublist])

# Step 2: Filter `caspian_IV` to exclude these `id2` values
df_filtered = df1[~df1['id'].isin(id2_to_remove)]

# Display the filtered DataFrame
print(df_filtered.info())

output_file = 'intersection_data_I-antro&I-data.xlsx'
with pd.ExcelWriter(output_file) as writer:
    result.to_excel(writer, index=False, sheet_name='ID2 Intersections')



<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, 14543 to 7332
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          82 non-null     float64 
 1   code        82 non-null     float64 
 2   sex         82 non-null     category
 3   university  82 non-null     object  
 4   state       82 non-null     float64 
 5   province    82 non-null     float64 
 6   region      72 non-null     category
 7   schoolType  82 non-null     category
 8   grade       82 non-null     category
 9   birthdat    82 non-null     float64 
 10  age         82 non-null     float64 
 11  fjob        81 non-null     category
 12  feducati    82 non-null     category
 13  mjob        82 non-null     category
 14  meducati    81 non-null     category
 15  househol    70 non-null     float64 
 16  housing     82 non-null     category
 17  weight      82 non-null     float64 
 18  height      82 non-null     float64 
 19  waist    

In [ ]:
# Returns a Series of True/False for each row, where True indicates a duplicate
duplicated_rows = processed_dfs['caspian_I_antro'].duplicated()

4172    False
4084    False
4085    False
4086    False
4087    False
        ...  
9144    False
9143    False
9142    False
9148    False
8937    False
Length: 20015, dtype: bool

In [9]:
processed_dfs['caspian_I_antro'].shape

(20015, 30)

In [10]:
processed_dfs['caspian_I_next'].shape

(20015, 30)

In [11]:
df_filtered.shape

(82, 30)